In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import skimage
from pathlib import Path
from scipy.io import loadmat
from scipy.interpolate import UnivariateSpline
import pickle
from copy import deepcopy

In [ ]:
from bayesee.perception import *
from bayesee.validation import *
from bayesee.visualization import *
from bayesee.generation import *
from bayesee.evaluation import *
from bayesee.iteration import *
from bayesee.partition import *

In [ ]:
%load_ext autoreload
%autoreload 2
plt.style.use('bayesee.academic')

In [ ]:
repo_path = Path.cwd().parents[0]
print(repo_path)

In [ ]:
file_name = repo_path / f"data/detection/phase-similarity/no-sync/models_AE.csv"
models_AE = pd.read_csv(file_name)

file_name = repo_path / f"data/detection/phase-similarity/no-sync/models_AZ.csv"
models_AZ = pd.read_csv(file_name)

file_name = repo_path / f"data/detection/phase-similarity/no-sync/models_HS.csv"
models_HS = pd.read_csv(file_name)

In [ ]:
file_name = repo_path / f"data/detection/phase-similarity/data_AE.csv"
data_AE = pd.read_csv(file_name)

file_name = repo_path / f"data/detection/phase-similarity/data_AZ.csv"
data_AZ = pd.read_csv(file_name)

file_name = repo_path / f"data/detection/phase-similarity/data_HS.csv"
data_HS = pd.read_csv(file_name)

In [ ]:
models_df = pd.concat([models_AE, models_AZ, models_HS], ignore_index=True)
data_df = pd.concat([data_AE, data_AZ, data_HS], ignore_index=True)

In [ ]:
array_simulation = models_df.columns
array_model = []
separator = "_100a"

for simulation in array_simulation:
    model, amplitude = simulation.split(separator)
    if model not in array_model:
        array_model.append(model)


print(array_model)

In [ ]:
data_df = recursive_bin(data_df, ["sa", "ss"], [2, 5])
array_unique_sa_bin = np.sort(data_df["level1_bin_sa"].unique())
array_unique_ss_bin = np.sort(data_df["level2_bin_ss"].unique())

In [ ]:
results = pd.DataFrame(
    columns=[
        "model",
        "sa_ave",
        "ss_ave",
        "amplitude",
        "dp",
    ]
)

for model in array_model:
    array_amplitude_string = []
    array_amplitude = []
    prefix = model + "_100a"

    for simulation in array_simulation:
        if simulation.startswith(prefix):
            amplitude_string = simulation[len(prefix) :]
            array_amplitude_string.append(amplitude_string)
            array_amplitude.append(int(amplitude_string) / 100.0)

    for index_amplitude, amplitude in enumerate(array_amplitude):
        for ss_bin in array_unique_ss_bin:
            bool_array_ss_bin = data_df["level2_bin_ss"] == ss_bin

            sa_ave = np.round(data_df["sa"][bool_array_ss_bin].mean(), 5)
            ss_ave = np.round(data_df["ss"][bool_array_ss_bin].mean(), 5)

            decision_variable = models_df.loc[
                bool_array_ss_bin, prefix + array_amplitude_string[index_amplitude]
            ]
            presence = data_df.loc[bool_array_ss_bin, "presence"]

            dva = decision_variable[presence == 0]
            dvb = decision_variable[presence == 1]
            na, nb = len(dva), len(dvb)
            dp = (dvb.mean() - dva.mean()) / np.sqrt(
                (na * dva.var() + nb * dvb.var()) / (na + nb)
            )

            results.loc[len(results)] = [
                model,
                sa_ave,
                ss_ave,
                amplitude,
                np.round(dp, 5),
            ]

In [ ]:
recursive_bin(results, ["sa_ave", "ss_ave"], [2, 5]).to_csv(
    repo_path / f"data/detection/phase-similarity/no-sync/models.csv",
    index=False,
)

In [ ]:
results = pd.read_csv(repo_path / f"data/detection/phase-similarity/no-sync/models.csv")

In [ ]:
array_unique_sa_bin = results["level1_bin_sa_ave"].unique()
array_unique_ss_bin = results["level2_bin_ss_ave"].unique()
array_model = np.sort(results["model"].unique())

In [ ]:
thresholds = pd.DataFrame(
    columns=[
        "model",
        "sa_ave",
        "ss_ave",
        "threshold",
    ]
)

array_color = [
    "#BD5500",
    "#0077BB",
    "#33BBEE",
    "#EE3377",
    "#009988",
    "#CC3311",
    "#BBBBBB",
]

for model in array_model:
    print(model)
    fig, axs = plt.subplots(nrows=1, ncols=len(array_unique_sa_bin), figsize=(14, 8))

    bool_array_model = results["model"] == model

    for index_sa_bin, sa_bin in enumerate(array_unique_sa_bin):
        bool_array_sa_bin = results["level1_bin_sa_ave"] == sa_bin
        for index_ss_bin, ss_bin in enumerate(
            np.sort(results["level2_bin_ss_ave"][bool_array_sa_bin].unique())
        ):
            bool_array_ss_bin = results["level2_bin_ss_ave"] == ss_bin

            bool_array_plot = bool_array_model & bool_array_ss_bin

            sort_index = np.argsort(results["amplitude"][bool_array_plot])
            amplitude = results["amplitude"][bool_array_plot].values[sort_index]
            dp = results["dp"][bool_array_plot].values[sort_index]
            spl = UnivariateSpline(amplitude, dp, ext=1)

            if index_sa_bin == 0:
                axs[index_sa_bin].scatter(
                    amplitude,
                    dp,
                    s=100,
                    c=array_color[ss_bin],
                    label=f"Bin {ss_bin}",
                )

                axs[index_sa_bin].plot(
                    amplitude, spl(amplitude), "--", lw=1, color=array_color[ss_bin]
                )

            elif index_sa_bin == 1:
                axs[index_sa_bin].scatter(
                    amplitude,
                    dp,
                    s=80,
                    facecolors="w",
                    linewidths=2,
                    edgecolors=array_color[ss_bin - len(array_unique_ss_bin) // 2],
                    label=f"Bin {ss_bin-len(array_unique_ss_bin)//2}",
                )

                axs[index_sa_bin].plot(
                    amplitude,
                    spl(amplitude),
                    "--",
                    lw=1,
                    color=array_color[ss_bin - len(array_unique_ss_bin) // 2],
                    zorder=-1,
                )
            f_amplitude = np.append(0, amplitude)
            f_dp = np.append(0, dp)
            th = UnivariateSpline(f_amplitude, f_dp - 1, ext=1).roots()
            threshold = 20 * np.log10(th.min())

            thresholds.loc[len(thresholds)] = [
                model,
                np.round(results.loc[bool_array_plot, "sa_ave"].values[0], 5),
                np.round(results.loc[bool_array_plot, "ss_ave"].values[0], 5),
                threshold,
            ]

        axs[index_sa_bin].legend()

    fig.text(0.5, 0, "Target amplitude", ha="center")
    fig.text(0, 0.5, "d'", va="center", rotation=90)
    fig.text(0.3, 0.9, "Low S$_A$", ha="center")
    fig.text(0.7, 0.9, "High S$_A$", ha="center")

    plt.show()

In [ ]:
recursive_bin(thresholds, ["sa_ave", "ss_ave"], [2, 5]).to_csv(
    repo_path / f"data/detection/phase-similarity/no-sync/model_thresholds.csv",
    index=False,
)

In [ ]:
thresholds = pd.read_csv(
    repo_path / f"data/detection/phase-similarity/no-sync/model_thresholds.csv"
)

In [ ]:
array_unique_sa_bin = np.sort(thresholds["level1_bin_sa_ave"].unique())
array_unique_ss_bin = np.sort(thresholds["level2_bin_ss_ave"].unique())
array_model = np.sort(thresholds["model"].unique())

In [ ]:
for model in array_model:
    print(model)
    fig, ax = plt.subplots()

    bool_array_model = thresholds["model"] == model

    for index_sa_bin, sa_bin in enumerate(array_unique_sa_bin):
        bool_array_sa_bin = thresholds["level1_bin_sa_ave"] == sa_bin

        bool_array_all = bool_array_model & bool_array_sa_bin

        ss_ave = thresholds.loc[bool_array_all, "ss_ave"]
        threshold = thresholds.loc[bool_array_all, "threshold"]

        if index_sa_bin == 0:
            ax.plot(ss_ave, threshold, linestyle="-", lw=5, c="k")
            ax.scatter(ss_ave, threshold, marker="o", s=320, c="k", label="Low S$_A$")

            y_min = np.round(threshold).min() - 1
            y_max = y_min + 27

        elif index_sa_bin == 1:
            ax.plot(ss_ave, threshold, linestyle="--", lw=5, c="k", zorder=-1)
            ax.scatter(
                ss_ave,
                threshold,
                marker="o",
                s=280,
                facecolors="w",
                lw=5,
                edgecolors="k",
                label="High S$_A$",
            )

            x_max = ss_ave.max() * 1.2

    ax.set(xlim=[-x_max, x_max], ylim=[y_min, y_max])

    ax.legend(loc="best")
    ax.set(xlabel="Image Similarity", ylabel="Threshold (dB)")

    plt.show()

In [ ]:
array_interest_model = ["UETM_v7", "UETM_v11", "UETM_v5"]
array_interest_label = ["$\sigma_0=32$", "$\sigma_0=160$", "$\sigma_0=320$"]

fig, ax = plt.subplots(figsize=(16, 16))
color_generator = iter(plt.cm.tab10(np.linspace(0, 1, len(array_interest_model))))

for index_model, model in enumerate(array_interest_model):
    bool_array_model = thresholds["model"] == model

    color = next(color_generator)

    for index_sa_bin, sa_bin in enumerate(array_unique_sa_bin):
        bool_array_sa_bin = thresholds["level1_bin_sa_ave"] == sa_bin

        bool_array_all = bool_array_model & bool_array_sa_bin

        ss_ave = thresholds.loc[bool_array_all, "ss_ave"]
        threshold = thresholds.loc[bool_array_all, "threshold"]

        if index_sa_bin == 0:
            ax.plot(ss_ave, threshold, linestyle="-", lw=5, color=color)
            ax.scatter(
                ss_ave,
                threshold,
                marker="o",
                s=320,
                color=color,
                label=array_interest_label[index_model],
            )

            y_min = np.round(threshold).min() - 1
            y_max = y_min + 27

        elif index_sa_bin == 1:
            ax.plot(ss_ave, threshold, linestyle="--", lw=5, color=color, zorder=-1)
            ax.scatter(
                ss_ave,
                threshold,
                marker="o",
                s=280,
                facecolors="w",
                lw=5,
                edgecolors=color,
            )

    ax.legend(loc="best")
    ax.set(xlabel="Image Similarity", ylabel="Threshold (dB)")

plt.show()